The next topic in the data preparation series is **Data Cleaning & Preparation - 2. Data Transformation**.

Data transformation is the process of modifying data to address issues identified during quality assessment and prepare it for analysis or modeling. This essential phase includes techniques for handling missing values, correcting data types, creating new features, scaling data, and encoding categorical variables.

Based on [`piplist.txt`](./README.md) output, we have robust libraries for this, including `pandas` (for core manipulation), `numpy`, and importantly, `scikit-learn` which provides excellent tools for preprocessing like imputation, scaling, and encoding.

**Step 1: Generate Sample Data for Transformation**

We'll generate data similar to the previous step but ensure it includes specific scenarios that require common transformations, like numerical columns with NaNs for imputation, categorical columns for encoding, and numerical columns for scaling.

In a new Excel cell, enter `=PY` and paste the following code, then press **Ctrl+Enter**:

In [ ]:
# Generate dummy data requiring various transformations
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import date, timedelta

fake = Faker()

num_records = 800

data = {
    'RecordID': range(1, num_records + 1),
    'CustomerName': [fake.name() for _ in range(num_records)],
    'PurchaseAmount': [round(random.uniform(10, 500), 2) if random.random() > 0.05 else np.nan for _ in range(num_records)], # Missing amounts
    'Rating': [random.choice([1, 2, 3, 4, 5, None]) for _ in range(num_records)], # Missing ratings
    'ProductCategory': [random.choice(['Electronics', 'Clothing', 'Home Goods', 'Groceries', 'Books', 'Other']) for _ in range(num_records)],
    'Region': [random.choice(['North', 'South', 'East', 'West', 'Central', 'Central']) for _ in range(num_records)], # Some imbalance
    'IsLoyaltyMember': [random.choice([True, False, None]) for _ in range(num_records)], # Boolean with missing
    'JoinDate': [fake.date_between(start_date='-5y', end_date='today') if random.random() > 0.03 else None for _ in range(num_records)], # Dates with missing
    'FeedbackText': [fake.sentence() if random.random() > 0.1 else '' for _ in range(num_records)], # Text, some empty
    'NumberOfVisits': [random.randint(1, 50) if random.random() > 0.04 else np.nan for _ in range(num_records)] # Missing visits
}

df_transform_raw = pd.DataFrame(data)

# Add some duplicate records
duplicate_records = df_transform_raw.sample(n=int(num_records * 0.03), replace=False)
df_transform_raw = pd.concat([df_transform_raw, duplicate_records], ignore_index=True)

# Add an outlier
df_transform_raw.loc[random.randint(0, len(df_transform_raw)-1), 'PurchaseAmount'] = 50000


# Shuffle for good measure
df_transform_raw = df_transform_raw.sample(frac=1, random_state=42).reset_index(drop=True)

df_transform_raw # Output the DataFrame

**Explanation:**

*   This code generates a DataFrame `df_transform_raw` with columns representing customer-related data.
*   It includes columns with:
    *   Missing numerical values (`PurchaseAmount`, `Rating`, `NumberOfVisits`).
    *   Categorical text (`ProductCategory`, `Region`).
    *   Boolean values (`IsLoyaltyMember`) with missing entries.
    *   Date values (`JoinDate`) with missing entries.
    *   Text data (`FeedbackText`).
    *   Duplicate rows.
    *   An artificial outlier in `PurchaseAmount`.
*   The result, `df_transform_raw`, will be spilled into your Excel sheet. Let's assume this data is placed in a range or Table named `DataForTransformation`.

**Step 2: Perform Data Transformations**

Now, we'll apply various transformations using `pandas` and `scikit-learn`.

In a **new** Excel cell, enter `=PY` and paste the following code. Replace `"DataForTransformation"` with the actual name of the Excel range/Table where your dummy data is. Press **Ctrl+Enter**.

In [ ]:
# Perform various data transformations
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer # For handling missing values
from sklearn.preprocessing import OneHotEncoder, StandardScaler # For encoding and scaling
from sklearn.compose import ColumnTransformer # To apply different transformers to different columns
from sklearn.pipeline import Pipeline # To chain transformations
from datetime import date, timedelta # Import timedelta

# Load the raw data from Excel
# IMPORTANT: Replace "DataForTransformation" with the actual name of your Excel range or Table
df_raw = xl("DataForTransformation[#All]", headers=True)
# Create a copy to work on, preserving the original raw data
df_transformed = df_raw.copy()


# --- Transformation 1: Handle Duplicate Rows ---
initial_rows = len(df_transformed)
df_transformed.drop_duplicates(inplace=True)
rows_after_dropping_duplicates = len(df_transformed)
duplicate_rows_removed = initial_rows - rows_after_dropping_duplicates


# --- Transformation 2: Handle Missing Values (Imputation) ---
# Identify columns for imputation
# Numeric columns: Impute with median (robust to outliers)
numeric_cols_for_imputation = ['PurchaseAmount', 'Rating', 'NumberOfVisits']
# Categorical column (Boolean): Impute with most frequent value
categorical_cols_for_imputation = ['IsLoyaltyMember'] # Note: Nones in bool might be read as object type

# Use Scikit-learn's SimpleImputer for a structured approach
# Create transformers for different column types
numeric_transformer = SimpleImputer(strategy='median')
categorical_transformer = SimpleImputer(strategy='most_frequent') # Works for object/string types too

# Define the columns to apply these transformations to
# Need to handle potential type issues from Excel import, especially for 'IsLoyaltyMember'
# Ensure 'IsLoyaltyMember' is treated as object/string before imputation
df_transformed['IsLoyaltyMember'] = df_transformed['IsLoyaltyMember'].astype(object) # Treat as generic object type for Imputer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols_for_imputation),
        ('cat', categorical_transformer, categorical_cols_for_imputation)
    ],
    remainder='passthrough' # Keep other columns (like RecordID, CustomerName, ProductCategory, Region, JoinDate, FeedbackText)
)

# Apply the imputation
# The output of ColumnTransformer is a numpy array, remember original column order/names
# Need to reconstruct DataFrame carefully, especially with 'remainder'
transformed_data_array = preprocessor.fit_transform(df_transformed)

# Get names of columns passed through + transformed columns
# preprocessor.get_feature_names_out() works in newer versions, fallback needed for older
# Manual tracking: original columns - imputed columns + imputed columns
remaining_cols = [col for col in df_transformed.columns if col not in numeric_cols_for_imputation + categorical_cols_for_imputation]
imputed_cols_order = numeric_cols_for_imputation + categorical_cols_for_imputation

# Reconstruct DataFrame
df_transformed_imputed = pd.DataFrame(transformed_data_array, columns=imputed_cols_order + remaining_cols)

# Ensure columns are in a logical order and types are correct after transformation
# ColumnTransformer might change column order and dtypes. Let's re-map and cast.
# Identify columns before and after transformation based on preprocessor
transformed_feature_names = numeric_cols_for_imputation + categorical_cols_for_imputation + remaining_cols # Assuming this is the order

# Reconstruct DataFrame with correct column names and attempt type casting
df_transformed_imputed = pd.DataFrame(transformed_data_array, columns=transformed_feature_names)

# Cast columns back to appropriate types after imputation and potential numpy conversion
for col in numeric_cols_for_imputation:
     df_transformed_imputed[col] = pd.to_numeric(df_transformed_imputed[col], errors='coerce') # Should be numeric after median imputation
df_transformed_imputed['IsLoyaltyMember'] = df_transformed_imputed['IsLoyaltyMember'].astype(bool) # Should be bool after most frequent imputation (assuming True/False were present)
# Re-convert Date column which might be object/string now
df_transformed_imputed['JoinDate'] = pd.to_datetime(df_transformed_imputed['JoinDate'], errors='coerce')


# --- Transformation 3: Correct/Convert Data Types ---
# Ensure numerical columns are numeric (done during imputation reconstruction, but double check)
# df_transformed_imputed['PurchaseAmount'] = pd.to_numeric(df_transformed_imputed['PurchaseAmount'], errors='coerce') # Already handled above
# df_transformed_imputed['Rating'] = pd.to_numeric(df_transformed_imputed['Rating'], errors='coerce') # Already handled above
# df_transformed_imputed['NumberOfVisits'] = pd.to_numeric(df_transformed_imputed['NumberOfVisits'], errors='coerce') # Already handled above
# Ensure Boolean column is boolean (handled above)
# Ensure Date column is datetime (handled above)
# Ensure Text column is string (usually read correctly, but can force)
df_transformed_imputed['FeedbackText'] = df_transformed_imputed['FeedbackText'].astype(str)


# --- Transformation 4: Feature Engineering (Example: Days Since Join) ---
# Calculate days since join date (relative to today, or a fixed date)
# Use a fixed analysis date for reproducibility
analysis_date = pd.to_datetime(date(2024, 6, 1)) # Convert analysis_date to datetime
df_transformed_imputed['DaysSinceJoin'] = (analysis_date - df_transformed_imputed['JoinDate']).dt.days # Use the .dt accessor after ensuring datetime type

# Handle cases where JoinDate was originally missing and is NaT (Not a Time)
df_transformed_imputed['DaysSinceJoin'] = df_transformed_imputed['DaysSinceJoin'].fillna(-1) # Use a sentinel value like -1


# --- Transformation 5: Handle Categorical Data (One-Hot Encoding) ---
# Select columns for one-hot encoding
categorical_cols_for_encoding = ['ProductCategory', 'Region']

# Use Scikit-learn's OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # handle_unknown='ignore' for unseen categories, sparse=False for dense array

# Fit and transform the selected columns
encoded_data = encoder.fit_transform(df_transformed_imputed[categorical_cols_for_encoding])

# Create a DataFrame from the encoded data with meaningful column names
encoded_col_names = encoder.get_feature_names_out(categorical_cols_for_encoding)
df_encoded = pd.DataFrame(encoded_data, columns=encoded_col_names, index=df_transformed_imputed.index)

# Drop the original categorical columns and join the new encoded columns
df_transformed_encoded = df_transformed_imputed.drop(columns=categorical_cols_for_encoding)
df_transformed_final = df_transformed_encoded.join(df_encoded)


# --- Transformation 6: Standardize Numerical Data ---
# Select numerical columns for standardization (exclude ID, engineered features if not needed)
# Exclude engineered 'DaysSinceJoin' for this example, or include it if needed
numerical_cols_for_scaling = ['PurchaseAmount', 'Rating', 'NumberOfVisits'] # Use the imputed columns

# Use Scikit-learn's StandardScaler
scaler = StandardScaler()

# Fit and transform the selected numerical columns
# Need to handle potential NaNs if imputation wasn't perfect, but SimpleImputer should have handled them
df_transformed_final[numerical_cols_for_scaling] = scaler.fit_transform(df_transformed_final[numerical_cols_for_scaling])


# --- Summarize Transformations ---
transformation_summary = pd.DataFrame({
    'Transformation': [
        'Duplicate Rows Removed',
        'Missing Values Imputed (Numeric: Median)',
        'Missing Values Imputed (Categorical: Most Frequent)',
        'Data Types Corrected',
        'Feature Engineered (Days Since Join)',
        'Categorical Columns One-Hot Encoded',
        'Numerical Columns Standardized'
    ],
    'Details': [
        duplicate_rows_removed,
        f"Columns: {', '.join(numeric_cols_for_imputation)}",
        f"Columns: {', '.join(categorical_cols_for_imputation)}",
        'Checked/Applied during imputation and reconstruction',
        'New column "DaysSinceJoin"',
        f"Columns: {', '.join(categorical_cols_for_encoding)}. Added {len(encoded_col_names)} new columns.",
        f"Columns: {', '.join(numerical_cols_for_scaling)}"
    ]
})


# Output the transformed DataFrame and the summary
output = {
    'Transformed Data Head': df_transformed_final.head(),
    'Transformed Data Info (dtypes, non-null counts)': df_transformed_final.info(verbose=False, memory_usage=False), # info doesn't return a df, just prints
    'Transformation Summary': transformation_summary,
    'Transformed Data Describe': df_transformed_final.describe(include='all') # describe is useful post-transformation
}

# For 'info', need to capture output if possible, or instruct user.
# Simple print will go to the console/error details. Let's use a string summary instead of info()
info_string = df_transformed_final.info(verbose=True, memory_usage=False, buf=None) # buf=None prints to stdout

# Let's add dtypes and non-null counts as DataFrames
dtypes_df = df_transformed_final.dtypes.reset_index()
dtypes_df.columns = ['Column', 'Dtype']

non_null_counts = df_transformed_final.notnull().sum().reset_index()
non_null_counts.columns = ['Column', 'NonNullCount']
non_null_counts['TotalRows'] = len(df_transformed_final)
non_null_counts['NullCount'] = non_null_counts['TotalRows'] - non_null_counts['NonNullCount']


output = {
    'Transformed Data Head': df_transformed_final.head(),
    'Transformed Data Dtypes': dtypes_df,
    'Transformed Data Non-Null Counts': non_null_counts,
    'Transformation Summary': transformation_summary,
    'Transformed Data Describe': df_transformed_final.describe(include='all')
}


output # Output the dictionary

**Explanation:**

*   We load the raw data (`df_raw`) and create a copy (`df_transformed`) to avoid modifying the original Excel data.
*   **Handle Duplicates:** `drop_duplicates()` is used to remove identical rows.
*   **Handle Missing Values:** `sklearn.impute.SimpleImputer` is used.
    *   We impute numerical columns (`PurchaseAmount`, `Rating`, `NumberOfVisits`) with the median value.
    *   We impute the `IsLoyaltyMember` column (which might be read as object/string due to `None`) with the most frequent value.
    *   `ColumnTransformer` is used to apply different imputation strategies to different subsets of columns, while `remainder='passthrough'` keeps other columns untouched during this specific step.
    *   We reconstruct the DataFrame after imputation, paying attention to column order and type casting.
*   **Correct Data Types:** Explicit `astype()` or `pd.to_numeric`/`pd.to_datetime` with `errors='coerce'` is used to ensure columns have the appropriate data types after transformations, as scikit-learn can sometimes return numpy arrays with generic types.
*   **Feature Engineering:** A new column `DaysSinceJoin` is created by calculating the difference between a fixed analysis date and the `JoinDate`. We handle missing `JoinDate` values by assigning a sentinel value (-1).
*   **Categorical Encoding:** `sklearn.preprocessing.OneHotEncoder` is used to convert categorical columns (`ProductCategory`, `Region`) into numerical format suitable for many models. Each unique value becomes a new binary column (one-hot encoding). `handle_unknown='ignore'` prevents errors if a new category appears later. `sparse_output=False` (or `sparse=False` in older versions) ensures the output is a dense numpy array.
*   **Numerical Scaling:** `sklearn.preprocessing.StandardScaler` is used to standardize selected numerical columns (`PurchaseAmount`, `Rating`, `NumberOfVisits`). Standardization scales data to have a mean of 0 and a standard deviation of 1, which is important for distance-based algorithms like K-Means or SVMs.
*   We generate a summary DataFrame detailing the transformations performed and return various views of the final `df_transformed_final` DataFrame, including its head, data types, non-null counts, and a summary description.

**Viewing the Output:**

*   Click the Python cell, then click the Python icon/button next to the formula bar.
*   Select "Excel Value" (**Ctrl+Shift+Alt+M**) for the DataFrames within the output dictionary ('Transformed Data Head', 'Transformed Data Dtypes', 'Transformed Data Non-Null Counts', 'Transformation Summary', 'Transformed Data Describe') to spill them into your sheet.

The output DataFrame `df_transformed_final` is now cleaned and transformed, ready for analysis or modeling steps. The next topic in the series is ["Data Cleaning & Preparation - 3. Data Integration"](./03-Data%20Cleaning%20&%20Preparation_03-Data%20Integration.md), which explores techniques for combining data from multiple sources while maintaining data quality.

**Further Analysis:**
* **Advanced Feature Engineering:** Using polynomial features, interaction terms, and custom transformers in scikit-learn pipelines
* **Automated Feature Selection:** Implementing LASSO, Ridge regression, or tree-based feature importance for dimensionality reduction
* **Custom Encoders:** Creating specialized encoders for domain-specific categorical variables or ordinal relationships
* **Advanced Imputation:** Using iterative imputation (MICE) or KNN imputation for complex missing data patterns
* **Transformation Validation:** Implementing cross-validation for transformation pipelines to ensure robustness